In [ ]:
Os dados desse estudo foram obtidos no data.rio , site da prefeitura do Rio de Janeiro. O   data.rio é composto por um vasto acervo de informações do Armazém de Dados, como mapas, tabelas e aplicações SIG Web melhorando e facilitando a administração da cidade, além de garantir maior visibilidade e interação com a população.

O data.rio é parte modelo de planejamento e gestão da informação pública da Prefeitura Municipal do Rio de Janeiro, iniciado em 2014 pela normatização do Sistema de Informações Urbanas (SIURB).

Para esse estudo inicial foi utilizada a 'Tabela 728 - Movimento médio diário de passageiros, segundo os transportes rodoviário, ferroviário, hidroviário e aeroviário no Município do Rio de Janeiro entre 1995-2019' disponível em .xsl (excel) no site data.rio na aba 'Transporte'.

Fonte: Superintendência Municipal de Transportes Urbanos - SMTU; Companhia do Metropolitano do Rio de Janeiro - Metrô-RJ; Opportrans Concessão Metroviária S.A.- METRÔ RIO; Companhia Fluminense de Trens Urbanos –  Flumitrens; SuperVia S.A.; Transporte Marítimo e Turismo S.A.- TRANSTUR; Barcas S.A.; Companhia de Navegação do Estado do Rio de Janeiro - CONERJ; Empresa Brasileira de Infraestrutura Aeroportuária - INFRAERO.					
Notas:					
Os bondes de Santa Teresa pararam de circular, em agosto de 2011, por causa de um grave acidente, voltando a funcionar em julho de 2015.					
... Dado numérico não disponível.					

No excel,

In [18]:
#importing the libraries
import pandas as pd

# data visualization
import seaborn as sns
sns.set(style="white")
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

In [19]:
#Getting the data
dados = pd.read_csv("Movimento_medio_transporte_RJ.csv")

In [20]:
#First look at the data
dados.head()

,Ano,Onibus,Metrô,Trem,Bonde,Hidroviário,Aeroviário
0,1995,3 292,323,266,"0,3",71,18
1,1996,3 261,352,223,"0,2",74,19
2,1997,3 222,269,158,"0,4",73,23
3,1998,3 242,358,130,"0,5",67,25
4,1999,2 968,399,175,"0,8",67,26
